In [ ]:
pip install porespy

In [3]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

import porespy as ps

import scipy.ndimage as spim
from skimage.measure import regionprops

# from porespy - seems like it's not a directly accessible function, but directly returns the chord lengths
def chord_counts(im):
    labels, N = spim.label(im > 0)
    props = regionprops(labels)
    chord_lens = np.array([i.filled_area for i in props])
    return chord_lens

def generate_chord(im):
    cld = {'count_x' : [], 'count_y' : []}

    crds_x = ps.filters.apply_chords(im=img, spacing=1, axis=0, trim_edges=False)
    crds_y = ps.filters.apply_chords(im=img, spacing=1, axis=1, trim_edges=False)
    cld['count_x'].append(chord_counts(crds_x))
    cld['count_y'].append(chord_counts(crds_y))

    return cld, crds_x, crds_y

ERROR:openpnm:PARDISO solver not installed, run `pip install pypardiso`. Otherwise, simulations will be slow. Apple M chips not supported.


## Chords generation
*   File structure should be directory/experiment groups/images
*   Images: There should be no other types of files (e.g. directory) other than masks




In [ ]:
#Edit your directory here
directory = ' '


os.chdir(directory)
# total_cld_info = {}
chords_list = []

# folder structure has to be: directory/experiment groups/images.tif
subfolders = glob('./*/')

for subfolder in subfolders:
    folderpath = os.path.join(directory, subfolder)

    files = os.listdir(folderpath)
    for filename in files:
      filepath = os.path.join(folderpath, filename)
      img = np.array(Image.open(filepath))

      print(filename)
      cld, crds_x, crds_y = generate_chord(img)

  ## This part is for demonstrating and can be skipped '''
      # plt.imshow(img, cmap = 'jet')

      # data_x = ps.metrics.chord_length_distribution(crds_x, bins=100)
      # data_y = ps.metrics.chord_length_distribution(crds_y, bins=100)

      # # This part print the image with horizontal and vertical chords
      # plt.subplots(1, 2, figsize =(10, 5))
      # plt.subplot(121)
      # plt.imshow(crds_x, interpolation='none')
      # plt.subplot(122)
      # plt.imshow(crds_y, interpolation='none')

      # # This part print the histogram of horizontal and vertical chords
      # plt.figure()
      # plt.bar(x=data_y.L, height=data_y.cdf, width=data_y.bin_widths, color='b', edgecolor='k', alpha=0.5, label = 'Y');
      # plt.legend()
      # plt.show()
      # plt.bar(x=data_x.L, height=data_x.cdf, width=data_x.bin_widths, color='r', edgecolor='k', alpha=0.5, label = 'X');
      # plt.legend(frameon = False)
      # plt.show()

  ## ''' Until here.

      chords_list = chords_list + list(cld['count_x'][0]) + list(cld['count_y'][0])
      # cld_name = str(filename)
      # total_cld_info[cld_name] = cld

  ## Save the information as .npy files. total_cld_info is just {'filename':'chords_list'}
  ## If you need to retract information from a certain image in later steps, you can use total_cld_info

    # np.save(directory + str('/' + data_group + 'total_cld.npy'), total_cld_info, allow_pickle = True)
    experiment_group = str(subfolder)[2:-1]
    np.save(directory + str('/' + experiment_group + '.npy'), chords_list, allow_pickle = True)